In [ ]:
import os
class SysMain(object):
    def __init__(self, *args, **kwargs):
        self.config_file_path = kwargs.get('Config','Config.yml')
        import yaml
        with open('Config.yml', 'r', encoding='utf-8') as file:
            self.config = yaml.load(file, Loader=yaml.FullLoader)
    def generate_name_path_map(self,folder_paths):
        pgo = {}
        for folder_path in folder_paths:
            for filepath,dirnames,filenames in os.walk(folder_path):
                for filename in filenames:
                    if filename.endswith('.lnk') or filename.endswith('.exe'):
                        name = filename.split('.')[0]
                        flg = not any(key in name or key == name for key in self.config['SystemPath']['Programs']['Exclude']['Name'].split(',')+self.config['SystemPath']['Programs']['Exclude']['Path'].split(','))
                        if flg:
                            pgo[name] = os.path.join(filepath, filename)
        return pgo

In [ ]:
from tools.tool_config import generate_tools_desc,tools_desc

generate_tools_desc()

In [ ]:
tools_desc

In [ ]:
import yaml
with open('Config.yml', 'r', encoding='utf-8') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
config

In [ ]:
generate_tools_desc()

In [1]:
from tools.tool_config import generate_tools_desc
from ollama_qw import OllamaQW
qw = OllamaQW(model_name='qwen2.5:3b',tools=generate_tools_desc())

2025-01-15 02:23:27.161 | INFO     | ollama_qw:__init__:19 - [{'type': 'function', 'function': {'name': 'close_light', 'description': '关掉机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'open_light', 'description': '打开机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'run_software', 'description': '打开或者运行程序、软件、应用、脚本等。', 'parameters': {'type': 'object', 'properties': {'software_name': {'type': 'string', 'description': '需要打开、运行或者执行的软件程序或者脚本的名称', 'enum': ['Administrative Tools', 'File Explorer', 'Ollama', 'Sonic Suite Companion', 'WPS Office', 'LiveCaptions', 'Magnify', 'Narrator', 'On-Screen Keyboard', 'VoiceAccess', 'Blender 4', 'DirectX修复工具', 'GW-SOUL', 'Unwise', 'KOOK', 'LetsVPN', 'Overwolf', 'Valorant Tracker', 'Postman', 'Raspberry Pi Imager', 'OpenRGB', 'SakuraLauncher_8a49d589b8997c26e5fe8ca0c8308d23', 'Steam', 'Command Prompt', 'Control Pane

In [2]:

def main():
    while True:
        msg = input(">: ")
        if msg.lower() == 'exit':
            break
        res = qw.chat(msg)
        print(f"Alice: {res['content']}")
main()

2025-01-15 02:23:42.280 | INFO     | ollama_qw:chat:23 - input message: {'role': 'user', 'content': '打开QQ音乐'}
2025-01-15 02:23:43.985 | INFO     | ollama_qw:chat:30 - model response is: model='qwen2.5:3b' created_at='2025-01-14T18:23:43.9847329Z' done=True done_reason='stop' total_duration=1702784200 load_duration=1069245700 prompt_eval_count=257 prompt_eval_duration=166000000 eval_count=23 eval_duration=242000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='run_software', arguments={'software_name': 'QQ Music'}))])
2025-01-15 02:23:43.986 | INFO     | ollama_qw:chat:36 - function name: run_software
2025-01-15 02:23:43.986 | INFO     | ollama_qw:chat:37 - function args: {'software_name': 'QQ Music'}
2025-01-15 02:23:43.987 | INFO     | ollama_qw:chat:46 - tool response is: {'role': 'tool', 'name': 'run_software', 'content': None}


Running software QQ Music


2025-01-15 02:23:44.450 | INFO     | ollama_qw:chat:53 - model response is: model='qwen2.5:3b' created_at='2025-01-14T18:23:44.4488547Z' done=True done_reason='stop' total_duration=460590700 load_duration=15609300 prompt_eval_count=296 prompt_eval_duration=5000000 eval_count=40 eval_duration=432000000 message=Message(role='assistant', content='已经打开了QQ音乐，请问接下来有什么可以帮助你的吗？你可以告诉我歌曲名，我可以帮你播放。或者你是否想听其他类型的音乐？如果你有任何需要帮助的事情，请随时告诉我。', images=None, tool_calls=None)


Alice: 已经打开了QQ音乐，请问接下来有什么可以帮助你的吗？你可以告诉我歌曲名，我可以帮你播放。或者你是否想听其他类型的音乐？如果你有任何需要帮助的事情，请随时告诉我。


In [ ]:
import importlib
module_name = 'tools.tool_list'
module = importlib.import_module(module_name)
module